In [ ]:
def get_payload_to_json(request):
    tickets_payload = []

    print('Fetching tickets')
    for ticket_id in range(0, 520):
        print(ticket_id)
        description_url = f"https://{domain}.freshdesk.com/api/v2/tickets/{ticket_id}"
        response = requests.get(description_url, headers=headers, auth=auth)
    
        if response.status_code == 200:
            ticket_description = response.json()
        else:
            print(f"Ticket ID {ticket_id} not found (Status: {response.status_code}). Skipping...")
            continue

        payload = get_payload(ticket_description)

        if payload['status'] == 2: continue
        if payload['query_type'] == None: continue 
        if payload['specific_issue'] == None: continue

        parsed_conversations = get_all_conversations(ticket_id, ticket_description)
        full_text_conversation = parsed_conversations['full_text_conversation']

        problem = None
        resolution = None

        while problem is None or resolution is None:
            problem, resolution = get_problem_and_resolution(full_text_conversation)

        payload['problem'] = problem
        payload['resolution'] = resolution

        print(f'Problem: {problem}\n\nResolution: {resolution}')
        print('-----')

        tickets_payload.append(payload) 

        file_path = "documents.json"
        if os.path.exists(file_path):
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
        else:
            data = []

        # data.extend(tickets_payload)
        data.append(payload)

        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

    return render(request, 'json.html')
